In [4]:
#######################
# Elastic Net Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data/MixedModels'
InputDir = '%s/l1/kfold' % DataBaseDir
OutputDir = '%s/l2/kfold' % DataBaseDir
kfold = 5
TreeNum = 100
InputCols = ['lgb_l2_%s' % i for i in range(TreeNum)]
InputCols2 = ['lgb_l2', 'lgb_huber', 'lgb_fair', 'etr', 'en', 'knn', 'rf', 'xgb_rmse', 'lassolars', 'gbr_ls', 'gbr_huber', 'gbr_lad', 'rgf', 'rgf_sib']
strategy = 'en'
start_time = datetime.datetime.now()
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
join_keys = ['air_store_id', 'visit_date']
start = time.time()
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    for i in range(len(InputCols)):
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        if(i == 0):
            FoldValid = valid
            FoldHoldout = holdout
            FoldTest = test
        else:
            FoldValid[InputCols[i]] = valid[InputCols[i]]
            FoldHoldout[InputCols[i]] = holdout[InputCols[i]]
            FoldTest[InputCols[i]] = test[InputCols[i]]
    FoldInputDir = '../../data/l1/kfold/%s' % (fold)
    for i in range(len(InputCols2)):
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, InputCols2[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, InputCols2[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, InputCols2[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        FoldValid[InputCols2[i]] = valid[InputCols2[i]]
        FoldHoldout[InputCols2[i]] = holdout[InputCols2[i]]
        FoldTest[InputCols2[i]] = test[InputCols2[i]]
    ###
    FoldValid['fold'] = fold
    valid_dfs.append(FoldValid)
    holdout_dfs.append(FoldHoldout)
    test_dfs.append(FoldTest)
    end = time.time()
    print('fold %s done, time elapsed %ss' % (fold, (end - start)))
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('----------------- CHECK -------------')
print('\n============================ TrainData')
print(TrainData.head(5))
print('\n============================ HoldoutData')
print(holdout_dfs[0].head(5))
print('\n============================ TestData')
print(test_dfs[0].head(5))
print('-------------------------------------')
# sys.exit(1)
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
for fold in range(kfold):
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': holdout_dfs[fold],
        'test': test_dfs[fold]
    }
    col = [c for c in FoldData['train'].columns if c not in ['id', 'air_store_id', 'visit_date', 'visitors', 'hpg_store_id', 'fold']]
    # train
    model = linear_model.ElasticNet(alpha= 0.0001, l1_ratio= 0.2, max_iter= 400, tol= 1e-4, selection= 'random', random_state= 2017)
    model.fit(FoldData['train'][col].astype(np.float32, copy=False), FoldData['train']['visitors'].values.astype(np.float32, copy=False))
    # for valid
    FoldData['valid'][strategy] = model.predict(FoldData['valid'][col])
    rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
    cv_score += rmsle_valid
    # for holdout
    FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][col])
    rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
    holdout_score += rmsle_holdout
    # for test
    FoldData['test'][strategy] = model.predict(FoldData['test'][col])
    y_test_pred += FoldData['test'][strategy]

    print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    #### output
    FoldOutputDir = '%s/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in FoldData.keys():
        if(mod == 'train'):
            continue
        OutCols = []
        if(mod == 'test'):
            OutCols.append('id')
        OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
        OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
        OutFoldData = FoldData[mod][OutCols]
        OutFoldData.to_csv(OutputFile, index= False)
    print('saving for %sth fold data done.' % (fold))
    
y_test_pred /= kfold  # Average test set predictions
cv_score /= kfold # Average valid set predictions
holdout_score /= kfold # Average holdout set predictions

# Create submission file
sub = pd.DataFrame()
sub['id'] = test_dfs[0]['id']
sub['visitors'] = np.expm1(y_test_pred)
OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/l2/submit' % DataBaseDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
print('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))
os.system('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))

finish_time = datetime.datetime.now()
elapsed = (finish_time - start_time).seconds
print('\n======================')
print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, elapsed))
print('======================\n')

fold 0 done, time elapsed 9.731130838394165s
fold 1 done, time elapsed 19.39321804046631s
fold 2 done, time elapsed 29.04415726661682s
fold 3 done, time elapsed 38.68392848968506s
fold 4 done, time elapsed 48.283973693847656s
----------------- CHECK -------------

============================ TrainData
           air_store_id visit_date  visitors  lgb_l2_0  lgb_l2_1  lgb_l2_2  \
0  air_ba937bf13d40fb24 2016-01-15  3.401197  3.521467  3.136138  3.554459   
1  air_ba937bf13d40fb24 2016-01-29  3.295837  3.543300  3.228056  3.592054   
2  air_ba937bf13d40fb24 2016-01-30  1.945910  3.357661  3.333841  3.406665   
3  air_ba937bf13d40fb24 2016-02-10  3.496508  3.229475  3.235015  3.346402   
4  air_ba937bf13d40fb24 2016-02-13  2.197225  3.356503  3.297490  3.397697   

   lgb_l2_3  lgb_l2_4  lgb_l2_5  lgb_l2_6  ...        knn        rf  xgb_rmse  \
0  3.394433  3.562601  3.264874  3.488428  ...   2.541251  3.362491  3.417059   
1  3.394433  3.562601  3.293011  3.488428  ...   2.588251  3.4886

/home/joe/.local/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.493397, holdout score 0.484614, valid length 45371
saving for 0th fold data done.
fold 1: valid score 0.485097, holdout score 0.485537, valid length 45371
saving for 1th fold data done.
fold 2: valid score 0.488313, holdout score 0.484524, valid length 45371
saving for 2th fold data done.
fold 3: valid score 0.492507, holdout score 0.485411, valid length 45370
saving for 3th fold data done.
fold 4: valid score 0.486295, holdout score 0.485533, valid length 45370
saving for 4th fold data done.
zip ../../data/MixedModels/l2/submit/en_submit_2018-01-09.zip ../../data/MixedModels/l2/submit/en_submit_2018-01-09.csv

CV score 0.4891, Holdout score 0.4851, Elapsed time: 160.00s

